# <div align="center">3D Geometry</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


## 3D Rotation Group

the 3D rotation group, often denoted $SO(3)$, is the group of all rotations about the origin of three-dimensional Euclidean space $R^3$ under the operation of composition. By definition, a rotation about the origin is a transformation that preserves the origin, Euclidean distance (so it is an isometry), and orientation (i.e. handedness of space). Every non-trivial rotation is determined by its axis of rotation (a line through the origin) and its angle of rotation. Composing two rotations results in another rotation; every rotation has a unique inverse rotation; and the identity map satisfies the definition of a rotation. Owing to the above properties (along composite rotations' associative property), the set of all rotations is a group under composition. Rotations are not commutative (for example, rotating 90 digree in the x-y plane followed by 90 digree in the y-z plane is not the same as S followed by R), making it a nonabelian group. Moreover, the rotation group has a natural structure as a manifold for which the group operations are smoothly differentiable; so it is in fact a Lie group. It is compact and has dimension 3.

Rotations are linear transformations of $R^3$ and can therefore be represented by matrices once a basis of $R^3$ has been chosen. Specifically, if we choose an orthonormal basis of $R^3$, every rotation is described by an orthogonal 3×3 matrix (i.e. a 3×3 matrix with real entries which, when multiplied by its transpose, results in the identity matrix) with determinant 1. The group SO(3) can therefore be identified with the group of these matrices under matrix multiplication. These matrices are known as ***special orthogonal matrices***, explaining the notation SO(3).

## Axis–Angle Representation

The axis–angle representation of a rotation parameterizes a rotation in a three-dimensional Euclidean space by two quantities: a unit vector ***$e$*** indicating the direction of an axis of rotation, and an angle $\theta$ describing the magnitude of the rotation about the axis. Only two numbers, not three, are needed to define the direction of a unit vector $e$ rooted at the origin because the magnitude of $e$ is constrained. For example, the elevation and azimuth angles of e suffice to locate it in any particular Cartesian coordinate frame.

***Example***: Say you are standing on the ground and you pick the direction of gravity to be the negative z direction. Then if you turn to your left, you will rotate 90 digrees about the z axis. Viewing the axis-angle representation as an ordered pair, this would be:

$(axis, angle) = ([e_x, e_y, e_z], \theta) = ([0, 0, 1], \pi / 2)$

The above example can be represented as a rotation vector with a magnitude of $\pi / 2$ pointing in the z direction: $[0, 0, \pi / 2]$


***Usage***: The axis–angle representation is convenient when dealing with rigid body dynamics. It is useful to both characterize rotations, and also for converting between different representations of rigid body motion, such as homogeneous transformations and twists.

## Rodrigues' Rotation Formula

A rotation matrix is an array of nine numbers. These are subject to the six norm and orthogonality constraints, so only three degrees of freedom are left: if three of the numbers are given, the other six can be computed from these equations. In numerical optimization problems, the redundancy of rotation matrices is inconvenient, and a minimal representation of rotation is often preferable.

The simplest such representation is based on Euler’s theorem, stating that ***every rotation can be described by an axis of rotation and an angle around it***. A compact representation of axis and angle is a three-dimensional rotation vector whose direction is the axis and whose magnitude is the angle in radians. The axis is oriented so that the acute-angle rotation is counterclockwise around it. As a consequence, the angle of rotation is always nonnegative, and at most $\pi$.

While simple, the rotation-vector representation of rotation must be used with some care. As defined earlier, the set of all rotation vectors is the three-dimensional ball of radius $\pi$. However, two antipodal points on the sphere, that is, two vectors r and −r with norm $\pi$, represent the same 180-degree rotation.

The formula for finding the rotation matrix corresponding to an angle-axis vector is called ***Rodrigues’ formula***. Let r be a rotation vector. If the vector is (0, 0, 0), then the rotation is zero, and the corresponding matrix is the identity matrix:

Let us now assume that r is not the zero vector. The unit vector for the axis of rotation is then $u=r/||r||$ and the angle is $\theta=||r||$. The rotation has no effect on a point p along the axis. Suppose then that p is off the axis. The rotated version of the vector p is $p' = Icos \theta + (1-cos \theta) u u^T + u_x sin \theta$

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def batch_skew(vec, batch_size=None):
    """
    vec is N x 3, batch_size is int

    returns N x 3 x 3. Skew_sym version of each matrix.
    """
    with tf.name_scope("batch_skew"):
        if batch_size is None:
            batch_size = vec.shape[0]
        col_inds = tf.constant([1, 2, 3, 5, 6, 7])
        indices = tf.reshape(
            tf.reshape(tf.range(0, batch_size) * 9, [-1, 1]) + col_inds,
            [-1, 1])
        updates = tf.reshape(
            tf.stack(
                [
                    -vec[:, 2], vec[:, 1], vec[:, 2], -vec[:, 0], -vec[:, 1],
                    vec[:, 0]
                ],
                axis=1), [-1])
        out_shape = [batch_size * 9]
        res = tf.scatter_nd(indices, updates, out_shape)
        res = tf.reshape(res, [batch_size, 3, 3])

        return res

In [3]:
def batch_rodrigues(theta, name=None):
    """
    Theta is N x 3
    """
    # with tf.name_scope(name, "batch_rodrigues", [theta]):
    with tf.name_scope(name, "batch_rodrigues"):
        batch_size = theta.shape[0]

        angle = tf.expand_dims(tf.norm(theta + 1e-8, axis=1), -1)
        r = tf.expand_dims(tf.div(theta, angle), -1)

        angle = tf.expand_dims(angle, -1)
        cos = tf.cos(angle)
        sin = tf.sin(angle)

        outer = tf.matmul(r, r, transpose_b=True, name="outer")

        eyes = tf.tile(tf.expand_dims(tf.eye(3), 0), [batch_size, 1, 1])
        R = cos * eyes + (1 - cos) * outer + sin * batch_skew(r, batch_size=batch_size)
        return R